--- Day 14: Docking Data ---

As your ferry approaches the sea port, the captain asks for your help again. The computer system that runs this port isn't compatible with the docking program on the ferry, so the docking parameters aren't being correctly initialized in the docking program's memory.

After a brief inspection, you discover that the sea port's computer system uses a strange bitmask system in its initialization program. Although you don't have the correct decoder chip handy, you can emulate it in software!

The initialization program (your puzzle input) can either update the bitmask or write a value to memory. Values and memory addresses are both 36-bit unsigned integers. For example, ignoring bitmasks for a moment, a line like mem[8] = 11 would write the value 11 to memory address 8.

The bitmask is always given as a string of 36 bits, written with the most significant bit (representing 2^35) on the left and the least significant bit (2^0, that is, the 1s bit) on the right. The current bitmask is applied to values immediately before they are written to memory: a 0 or 1 overwrites the corresponding bit in the value, while an X leaves the bit in the value unchanged.

For example, consider the following program:

```
mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
mem[8] = 11
mem[7] = 101
mem[8] = 0
```

This program starts by specifying a bitmask (mask = ....). The mask it specifies will overwrite two bits in every written value: the 2s bit is overwritten with 0, and the 64s bit is overwritten with 1.

The program then attempts to write the value 11 to memory address 8. By expanding everything out to individual bits, the mask is applied as follows:

```
value:  000000000000000000000000000000001011  (decimal 11)
mask:   XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
result: 000000000000000000000000000001001001  (decimal 73)
```

So, because of the mask, the value 73 is written to memory address 8 instead. Then, the program tries to write 101 to address 7:

```
value:  000000000000000000000000000001100101  (decimal 101)
mask:   XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
result: 000000000000000000000000000001100101  (decimal 101)
```

This time, the mask has no effect, as the bits it overwrote were already the values the mask tried to set. Finally, the program tries to write 0 to address 8:

```
value:  000000000000000000000000000000000000  (decimal 0)
mask:   XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
result: 000000000000000000000000000001000000  (decimal 64)
```

64 is written to address 8 instead, overwriting the value that was there previously.

To initialize your ferry's docking program, you need the sum of all values left in memory after the initialization program completes. (The entire 36-bit address space begins initialized to the value 0 at every address.) In the above example, only two values in memory are not zero - 101 (at address 7) and 64 (at address 8) - producing a sum of 165.

Execute the initialization program. What is the sum of all values left in memory after it completes?


In [1]:
#!fsharp
let testInput = """mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
mem[8] = 11
mem[7] = 101
mem[8] = 0"""

let taskInput = System.IO.File.ReadAllText "task.txt"

In [1]:
#!fsharp
type Instruction = Mask of uint64 * uint64
                  | Write of uint64 * uint64

open System.Text.RegularExpressions

let (|Regex|_|) (p:string) (s:string)  =
    let m = Regex.Match(s, p)
    if m.Success then Some(List.tail [ for g in m.Groups -> g.Value ])
    else None

let parse (input: string) : seq<Instruction> =
    input.Replace("\r", "").Split("\n")
    |> Seq.map (
            function
            | Regex @"mask\s*=\s*([01X]+)" [mask] -> 
                let mutable andMask = 0UL
                let mutable orMask = 0UL
                Seq.iter (
                    fun bit ->
                        andMask <- andMask <<< 1
                        orMask <- orMask <<< 1
                        if bit = '1' then
                            orMask <- orMask ||| 1UL
                        elif bit = 'X' then
                            andMask <- andMask ||| 1UL
                ) mask
                Mask(andMask, orMask)
            | Regex @"mem\[(\d+)\]\s+=\s+(\d+)" [address; value] ->
                Write(uint64 address, uint64 value)
            | unknown -> 
                failwith ("Unknown instruction " + unknown)
        )

testInput |> parse

index,type,Item1,Item2
0,FSI_0090+Instruction+Mask,68719476669,64
1,FSI_0090+Instruction+Write,8,11
2,FSI_0090+Instruction+Write,7,101
3,FSI_0090+Instruction+Write,8,0


In [1]:
#!fsharp
let execute (program: seq<Instruction>) =
    let mutable andMask = 0UL
    let mutable orMask = 0UL
    let mutable memory: Map<uint64, uint64> = Map.empty
    program
    |> Seq.iter (
        function 
        | Mask (newAndMask, newOrMask) ->
            andMask <- newAndMask
            orMask <- newOrMask
        | Write (target, value) ->
            let newValue = (value &&& andMask) ||| orMask
            memory <- memory.Add(target, newValue)
    )
    memory

testInput |> parse |> execute

key,value
7,101
8,64


In [1]:
#!fsharp
let toResult (memory: Map<uint64,uint64>) =
    Map.fold (
        fun acc key value ->
            acc + value
    ) 0UL memory

testInput |> parse |> execute |> toResult

165

In [1]:
#!fsharp
taskInput |> parse |> execute |> toResult

14954914379452

--- Part Two ---

For some reason, the sea port's computer system still can't communicate with your ferry's docking program. It must be using version 2 of the decoder chip!

A version 2 decoder chip doesn't modify the values being written at all. Instead, it acts as a memory address decoder. Immediately before a value is written to memory, each bit in the bitmask modifies the corresponding bit of the destination memory address in the following way:

*    If the bitmask bit is 0, the corresponding memory address bit is unchanged.
*    If the bitmask bit is 1, the corresponding memory address bit is overwritten with 1.
*    If the bitmask bit is X, the corresponding memory address bit is floating.

A floating bit is not connected to anything and instead fluctuates unpredictably. In practice, this means the floating bits will take on all possible values, potentially causing many memory addresses to be written all at once!

For example, consider the following program:

```
mask = 000000000000000000000000000000X1001X
mem[42] = 100
mask = 00000000000000000000000000000000X0XX
mem[26] = 1
```

When this program goes to write to memory address 42, it first applies the bitmask:

```
address: 000000000000000000000000000000101010  (decimal 42)
mask:    000000000000000000000000000000X1001X
result:  000000000000000000000000000000X1101X
```

After applying the mask, four bits are overwritten, three of which are different, and two of which are floating. Floating bits take on every possible combination of values; with two floating bits, four actual memory addresses are written:

```
000000000000000000000000000000011010  (decimal 26)
000000000000000000000000000000011011  (decimal 27)
000000000000000000000000000000111010  (decimal 58)
000000000000000000000000000000111011  (decimal 59)
```

Next, the program is about to write to memory address 26 with a different bitmask:

```
address: 000000000000000000000000000000011010  (decimal 26)
mask:    00000000000000000000000000000000X0XX
result:  00000000000000000000000000000001X0XX
```

This results in an address with three floating bits, causing writes to eight memory addresses:

```
000000000000000000000000000000010000  (decimal 16)
000000000000000000000000000000010001  (decimal 17)
000000000000000000000000000000010010  (decimal 18)
000000000000000000000000000000010011  (decimal 19)
000000000000000000000000000000011000  (decimal 24)
000000000000000000000000000000011001  (decimal 25)
000000000000000000000000000000011010  (decimal 26)
000000000000000000000000000000011011  (decimal 27)
```

The entire 36-bit address space still begins initialized to the value 0 at every address, and you still need the sum of all values left in memory at the end of the program. In this example, the sum is 208.

Execute the initialization program using an emulator for a version 2 decoder chip. What is the sum of all values left in memory after it completes?


In [1]:
#!fsharp
let parse2 (input: string) : seq<Instruction> =
    input.Replace("\r", "").Split("\n")
    |> Seq.map (
            function
            | Regex @"mask\s*=\s*([01X]+)" [mask] -> 
                let mutable xMask = 0UL
                let mutable orMask = 0UL
                Seq.iter (
                    fun bit ->
                        xMask <- xMask <<< 1
                        orMask <- orMask <<< 1
                        if bit = '1' then
                            orMask <- orMask ||| 1UL
                        elif bit = 'X' then
                            xMask <- xMask ||| 1UL
                ) mask
                Mask(xMask, orMask)
            | Regex @"mem\[(\d+)\]\s+=\s+(\d+)" [address; value] ->
                Write(uint64 address, uint64 value)
            | unknown -> 
                failwith ("Unknown instruction " + unknown)
        )

In [1]:
#!fsharp
let testInput2 = """mask = 000000000000000000000000000000X1001X
mem[42] = 100
mask = 00000000000000000000000000000000X0XX
mem[26] = 1"""

testInput2 |> parse2

index,type,Item1,Item2
0,FSI_0090+Instruction+Mask,33,18
1,FSI_0090+Instruction+Write,42,100
2,FSI_0090+Instruction+Mask,11,0
3,FSI_0090+Instruction+Write,26,1


In [1]:
#!fsharp
// resolve the instructions into masked set of writes
let forwardRun program =
    let mutable xMask = 0UL
    let mutable orMask = 0UL
    program
    |> Seq.map (
        function 
        | Mask (newXMask, newOrMask) ->
            xMask <- newXMask
            orMask <- newOrMask
            None
        | Write (target, value) ->
            let newTarget = (target ||| orMask) &&& (~~~xMask)
            Some (newTarget, xMask, value)
    )
    |> Seq.choose id

testInput2 |> parse2 |> forwardRun

index,Item1,Item2,Item3
0,26,33,100
1,16,11,1


In [1]:
#!fsharp
let countBits (mask: uint64) =
    let rec inner sum mask =
        if mask = 0UL then
            sum
        elif mask &&& 1UL <>0UL then
            inner (sum+1) (mask >>> 1)
        else
            inner sum (mask >>> 1)
    inner 0 mask

countBits 35UL

3

In [1]:
#!fsharp
// resolve the writes into independent non-overlaping groups
let resolveResults (partial: seq<uint64 * uint64 * uint64>) =
    partial
    |> Seq.fold (
        fun acc entry ->
            let mutable target, xMask, value = entry
            let result = 
                List.map (
                    fun (prevTarget, prevXMask, prevValue) ->
                        // get the fixed bits outside X masks
                        let prevAddress = prevTarget &&& ~~~( prevXMask ||| xMask )
                        let address = target &&& ~~~(prevXMask ||| xMask)
                        // the entries interact only if the fixed bits exactly match
                        if prevAddress = address then
                            // if previous target had no floating bits, or they exactly overlap, then it's all gone
                            if  prevXMask = 0UL || prevXMask = xMask  then
                                []
                            else
                                let mutable subset = List.empty
                                let mutable forcedMask = 0UL
                                let mutable forcedAddress = 0UL
                                for bit in 0..35 do
                                    let testMask = 1UL <<< bit
                                    // if this is fixed bit in the new entry, but variable in previous, we can generate subset
                                    if (xMask &&& testMask) = 0UL && (prevXMask &&& testMask) <> 0UL then
                                        forcedMask <- forcedMask ||| testMask
                                        // generate address with the bit set to opposite to target
                                        forcedAddress <- forcedAddress ||| ( ~~~target &&& testMask)
                                        subset <- subset @ [( (prevTarget &&& ~~~forcedMask) ||| forcedAddress, prevXMask &&& ~~~forcedMask, prevValue  )]
                                        // and set the bit back to matching for further tests
                                        forcedAddress <- forcedAddress ^^^ testMask
                                subset
                        else
                            [(prevTarget, prevXMask, prevValue)]
                ) acc
                |> List.concat
            result @ [(target, xMask, value)]
    ) List.empty 

let prettyPrint values =
    values
    |> Seq.map 
        ( fun (target, mask, v) -> 
            let mutable s = ""
            let mutable found = false
            for i in 35..-1..0 do
                if i = 7 then found <- true
                let t = 1UL <<< i
                if t &&& mask <> 0UL then
                    s <- s + "X"
                    found <- true
                elif t &&& target <> 0UL then
                    s <- s + "1"
                    found <- true
                elif found then
                    s <- s + "0"
            (s, v)
        )

[(0UL, 7UL, 15UL); (2UL, 0UL, 16UL)] |> resolveResults |> prettyPrint

index,Item1,Item2
0,00000XX1,15
1,00000X00,15
2,00000110,15
3,00000010,16


In [1]:
#!fsharp
testInput2 |> parse2 |> forwardRun |> resolveResults |> prettyPrint

index,Item1,Item2
0,0011101X,100
1,0001X0XX,1


In [1]:
#!fsharp
testInput2 |> parse2 |> forwardRun

index,Item1,Item2,Item3
0,26,33,100
1,16,11,1


In [1]:
#!fsharp
let calculateResult values = 
    values
    |> Seq.fold 
        (fun acc (target, mask, v) ->
            acc + v * (1UL <<< countBits mask)
        )    
        0UL

testInput2 |> parse2 |> forwardRun |> resolveResults |> calculateResult


208

In [1]:
#!fsharp
taskInput |> parse2 |> forwardRun |> resolveResults |> calculateResult

3415488160714